<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%B3%D8%AA%D8%B1%D8%A7%D8%AA%D9%8A%D8%AC%D9%8A%D8%A9_mt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['high'], data['low'], data['close'], length=14)
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# مثال على كيفية استخدام الدوال
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)

        # بناء الاستراتيجية
        build_strategy(gold_data)

# إغلاق الاتصال
mt5.shutdown()


In [ ]:
!pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        price = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "sl": 0,  # Stop Loss
            "tp": 0,  # Take Profit
            "deviation": 10,
            "magic": 234000,  # Magic number
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print("Order placed successfully!")
    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['high'], data['low'], data['close'], length=14)
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # إشارة التداول
        if data['MA'].iloc[-1] > data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أكبر من السابق
            place_order("XAUUSD", mt5.ORDER_BUY, volume=0.1)  # شراء
        elif data['MA'].iloc[-1] < data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أقل من السابق
            place_order("XAUUSD", mt5.ORDER_SELL, volume=0.1)  # بيع

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# مثال على كيفية استخدام الدوال
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)

        # بناء الاستراتيجية
        build_strategy(gold_data)

# إغلاق الاتصال
mt5.shutdown()


In [ ]:
!pip install MetaTrader5 pandas-ta requests beautifulsoup4 textblob matplotlib scikit-learn xgboost
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        price = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "sl": 0,  # Stop Loss
            "tp": 0,  # Take Profit
            "deviation": 10,
            "magic": 234000,  # Magic number
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print("Order placed successfully!")
    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['high'], data['low'], data['close'], length=14)
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # إشارة التداول
        if data['MA'].iloc[-1] > data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أكبر من السابق
            place_order("XAUUSD", mt5.ORDER_BUY, volume=0.1)  # شراء
        elif data['MA'].iloc[-1] < data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أقل من السابق
            place_order("XAUUSD", mt5.ORDER_SELL, volume=0.1)  # بيع

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        build_strategy(gold_data)

# إنهاء الاتصال بـ MT5
mt5.shutdown()


In [ ]:
#!pip install MetaTrader5 pandas-ta requests beautifulsoup4 textblob matplotlib scikit-learn xgboost
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        price = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "sl": 0,  # Stop Loss
            "tp": 0,  # Take Profit
            "deviation": 10,
            "magic": 234000,  # Magic number
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print("Order placed successfully!")
    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.RSI(data['close'], length=period)  # استخدم RSI بدلاً من rsi
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.SMA(data['close'], length=period)  # استخدم SMA بدلاً من sma
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.MOM(data['close'], length=period)  # استخدم MOM بدلاً من mom
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()  # تأكد من أن العمود 'volume' موجود
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.ATR(data['high'], data['low'], data['close'], length=14)  # استخدم ATR بدلاً من atr
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # إشارة التداول
        if data['MA'].iloc[-1] > data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أكبر من السابق
            place_order("XAUUSD", mt5.ORDER_BUY, volume=0.1)  # شراء
        elif data['MA'].iloc[-1] < data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أقل من السابق
            place_order("XAUUSD", mt5.ORDER_SELL, volume=0.1)  # بيع

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        build_strategy(gold_data)

# إنهاء الاتصال بـ MT5
mt5.shutdown()


In [ ]:
!pip install MetaTrader5 requests beautifulsoup4 textblob matplotlib scikit-learn xgboost
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        price = mt5.symbol_info_tick(symbol).ask if order_type == mt5.ORDER_BUY else mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "sl": 0,  # Stop Loss
            "tp": 0,  # Take Profit
            "deviation": 10,
            "magic": 234000,  # Magic number
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print("Order placed successfully!")
    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()  # تأكد من أن العمود 'volume' موجود
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        high_low = data['high'] - data['low']
        high_close = (data['high'] - data['close'].shift(1)).abs()
        low_close = (data['low'] - data['close'].shift(1)).abs()
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        data['ATR'] = tr.rolling(window=14).mean()
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # إشارة التداول
        if data['MA'].iloc[-1] > data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أكبر من السابق
            place_order("XAUUSD", mt5.ORDER_BUY, volume=0.1)  # شراء
        elif data['MA'].iloc[-1] < data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أقل من السابق
            place_order("XAUUSD", mt5.ORDER_SELL, volume=0.1)  # بيع

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        build_strategy(gold_data)

# إنهاء الاتصال بـ MT5
mt5.shutdown()


In [ ]:
#!pip install MetaTrader5 requests beautifulsoup4 textblob matplotlib scikit-learn xgboost
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = "NU6-caQp"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_H1, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)
        df.drop(columns=['spread', 'real_volume'], inplace=True, errors='ignore')
        return df[['close', 'high', 'low', 'volume']]  # تأكد من تضمين العمود 'volume'
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def place_order(symbol, order_type, volume):
    """Place an order in MetaTrader 5."""
    try:
        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": mt5.ORDER_BUY if order_type == "buy" else mt5.ORDER_SELL,  # استخدام القيم الصحيحة
            "price": price,
            "sl": 0,  # Stop Loss
            "tp": 0,  # Take Profit
            "deviation": 10,
            "magic": 234000,  # Magic number
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,  # Good till canceled
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print("Order placed successfully!")
    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['close'].iloc[i-1] and data['close'].iloc[i-1] < data['close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
            elif data['close'].iloc[i] < data['close'].iloc[i-1] and data['close'].iloc[i-1] > data['close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['close'].iloc[i] > data['high'].iloc[i-1]:
                supply_zones.append((data.index[i], data['high'].iloc[i-1]))
            elif data['close'].iloc[i] < data['low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['low'].rolling(window=20).min()
        resistance = data['high'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        delta = data['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        data['RSI'] = 100 - (100 / (1 + rs))
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = data['close'].rolling(window=period).mean()
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = data['close'].diff(periods=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['volume'].rolling(window=20).mean()  # تأكد من أن العمود 'volume' موجود
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['close'].iloc[i] < data['close'].iloc[i-1] and
                data['close'].iloc[i+1] < data['close'].iloc[i] and
                data['close'].iloc[i+2] > data['close'].iloc[i+1] and
                data['close'].iloc[i+3] < data['close'].iloc[i+2] and
                data['close'].iloc[i+4] > data['close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['close'].pct_change()
        data['Log_Returns'] = np.log(data['close'] / data['close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['high'].rolling(window=window).max()
        data['Channel_Lower'] = data['low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        high_low = data['high'] - data['low']
        high_close = (data['high'] - data['close'].shift(1)).abs()
        low_close = (data['low'] - data['close'].shift(1)).abs()
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        data['ATR'] = tr.rolling(window=14).mean()
        data['Stop_Loss'] = data['close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['close'].iloc[i] < البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] > البيانات['close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))
            elif البيانات['close'].iloc[i] > البيانات['close'].iloc[i-1] and البيانات['close'].iloc[i-1] < البيانات['close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'close', 'high', 'low' إلى أرقام
        for col in ['close', 'high', 'low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # إشارة التداول
        if data['MA'].iloc[-1] > data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أكبر من السابق
            place_order("XAUUSD", "buy", volume=0.1)  # شراء
        elif data['MA'].iloc[-1] < data['MA'].iloc[-2]:  # إذا كان المتوسط المتحرك الحالي أقل من السابق
            place_order("XAUUSD", "sell", volume=0.1)  # بيع

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices()
    if gold_data is not None:
        build_strategy(gold_data)

# إنهاء الاتصال بـ MT5
mt5.shutdown()
